In [4]:
import sys
sys.path.append("../classes")

In [5]:
import pickle
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

import numpy as np, pandas as pd
from geno_classifier import *
from geno_utils import parse_database
from itertools import starmap

import GEOparse

In [8]:
diabetes_dataset = GEOparse.get_GEO(filepath='../datasets/GDS3782_full.soft').table

01-Feb-2018 12:18:12 INFO GEOparse - Parsing ../datasets/GDS3782_full.soft: 
01-Feb-2018 12:18:12 DEBUG GEOparse - DATABASE: Geo
01-Feb-2018 12:18:12 DEBUG GEOparse - DATASET: GDS3782
01-Feb-2018 12:18:12 DEBUG GEOparse - SUBSET: GDS3782_1
01-Feb-2018 12:18:12 DEBUG GEOparse - SUBSET: GDS3782_2
01-Feb-2018 12:18:12 DEBUG GEOparse - ANNOTATION: 
01-Feb-2018 12:18:12 ERROR GEOparse - Cannot recognize type Annotation
01-Feb-2018 12:18:12 DEBUG GEOparse - DATASET: GDS3782
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:88 DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.


In [9]:
healthy_raw = """
#GSM524151 = Value for GSM524151: beta-cells_non-diabetic condition_donor1; src: human beta-cells_non-diabetic
#GSM524152 = Value for GSM524152: beta-cells_non-diabetic condition_donor2; src: human beta-cells_non-diabetic
#GSM524153 = Value for GSM524153: beta-cells_non-diabetic condition_donor3; src: human beta-cells_non-diabetic
#GSM524154 = Value for GSM524154: beta-cells_non-diabetic condition_donor4; src: human beta-cells_non-diabetic
#GSM524155 = Value for GSM524155: beta-cells_non-diabetic condition_donor5; src: human beta-cells_non-diabetic
#GSM524156 = Value for GSM524156: beta-cells_non-diabetic condition_donor6; src: human beta-cells_non-diabetic
#GSM524157 = Value for GSM524157: beta-cells_non-diabetic condition_donor7; src: human beta-cells_non-diabetic
#GSM524158 = Value for GSM524158: beta-cells_non-diabetic condition_donor8; src: human beta-cells_non-diabetic
#GSM524159 = Value for GSM524159: beta-cells_non-diabetic condition_donor9; src: human beta-cells_non-diabetic
#GSM524160 = Value for GSM524160: beta-cells_non-diabetic condition_donor10; src: human beta-cells_non-diabetic"""

In [10]:
unhealthy_raw = """
#GSM524161 = Value for GSM524161: beta-cells_diabetic condition_donor1; src: human beta-cells_diabetic
#GSM524162 = Value for GSM524162: beta-cells_diabetic condition_donor2; src: human beta-cells_diabetic
#GSM524163 = Value for GSM524163: beta-cells_diabetic condition_donor3; src: human beta-cells_diabetic
#GSM524164 = Value for GSM524164: beta-cells_diabetic condition_donor4; src: human beta-cells_diabetic
#GSM524165 = Value for GSM524165: beta-cells_diabetic condition_donor5; src: human beta-cells_diabetic
#GSM524166 = Value for GSM524166: beta-cells_diabetic condition_donor6; src: human beta-cells_diabetic
#GSM524167 = Value for GSM524167: beta-cells_diabetic condition_donor7; src: human beta-cells_diabetic
#GSM524168 = Value for GSM524168: beta-cells_diabetic condition_donor8; src: human beta-cells_diabetic
#GSM524169 = Value for GSM524169: beta-cells_diabetic condition_donor9; src: human beta-cells_diabetic
#GSM524170 = Value for GSM524170: beta-cells_diabetic condition_donor10; src: human beta-cells_diabetic
"""

In [11]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [12]:
len(healthy),len(unhealthy)

(10, 10)

In [13]:
# healthy = list(healthy)[:16]
# unhealthy = list(unhealthy)[:16]

# # pickle.dump({"helthy":healthy,
# #              "unhealthy":unhealthy},
# #            open("colon_cancer_subset","wb"))

In [14]:
healthy = set(healthy)
unhealthy = set(unhealthy)

In [15]:
sample_columns = list(healthy | unhealthy)

In [16]:
cle = diabetes_dataset[["ID_REF","IDENTIFIER"]+sample_columns]

In [17]:
cle.dropna(inplace=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
cle[cle.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM524166,GSM524158,GSM524163
0,1053_3p_at,RFC2,11.77,2.41,6.66
1,117_3p_at,HSPA6,48.63,37.28,50.87
2,1494_3p_f_at,CYP2A6,52.95,59.12,62.74
3,1552275_3p_s_at,PXK,17.09,6.56,7.17
4,1552281_3p_at,SLC39A5,49.57,32.01,67.00


In [19]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [21]:
X, y = parse_database(cle, labels=dict(labels_), n_jobs=-1)

In [ ]:
pickle.dump({"X":X,"y":y}, open("diabetes_Xy", "wb"))

In [ ]:
coca = pickle.load(open("colon_cancer_Xy", "rb"))
X = coca["X"]
y = coca["y"]

## FVA

In [6]:
results = flux_variance_analysis(X, y)
pickle.dump(results, open("diabetes.results", "wb"))

../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../c

cobra/util/solver.py:403 UserWarning: solver status is 'infeasible'
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
../classes/genobolitics.py:72 UserWarning: model returned the empty string as reaction rule!
../classes/genobolitics.py:56 UserWarning: could not evaluate boolean expression, objective-coeff is set to ZERO!
../classes/genobolitics.py:36 UserWarning: some operands are missing from logical expression!
/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
../classes/genobolitics.py:72 UserWarnin

/usr/local/lib/python3.5/dist-packages/optlang/cplex_interface.py:459 DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
cobra/util/solver.py:403 UserWarning: solver status is 'infeasible'
